In [1]:
!pip install pyspark
!pip install py4j
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark
from pyspark.context import SparkContext
import pyspark.sql
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col 
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import min, max, mean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 62.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3285d87878d7854c48e2132ac5b6e64a5a98ab157c1501faf72748e64dc386ad
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
demo = files.upload()

Saving DEMO.csv to DEMO.csv


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]") \
.appName("DEMO")  \
.config('spark.ui.port','4050') \
.getOrCreate()
spark

In [7]:
df = spark.read.csv('DEMO.csv')
df.show()

+---+--------------------+------------------+----+-------+------+-----+-------+--------------------+----+
|_c0|                 _c1|               _c2| _c3|    _c4|   _c5|  _c6|    _c7|                 _c8| _c9|
+---+--------------------+------------------+----+-------+------+-----+-------+--------------------+----+
|  1|Eldon Base for st...|Muhammed MacIntyre|   3|-213.25| 38.94|   35|Nunavut|Storage & Organiz...| 0.8|
|  2|"1.7 Cubic Foot C...|      Barry French| 293| 457.81|208.16|68.02|Nunavut|          Appliances|0.58|
|  3|Cardinal Slant-D�...|      Barry French| 293|  46.71|  8.69| 2.99|Nunavut|Binders and Binde...|0.39|
|  4|                R380|     Clay Rozendal| 483|1198.97|195.99| 3.99|Nunavut|Telephones and Co...|0.58|
|  5|Holmes HEPA Air P...|    Carlos Soltero| 515|  30.94| 21.78| 5.94|Nunavut|          Appliances| 0.5|
|  6|G.E. Longer-Life ...|    Carlos Soltero| 515|   4.43|  6.64| 4.95|Nunavut|  Office Furnishings|0.37|
|  7|Angle-D Binders w...|      Carl Jackson| 

In [7]:
schema=StructType([\
                  StructField("Number",IntegerType(),True),\
                  StructField("Product Name",StringType(),True),\
                  StructField("Person Name",StringType(),True),\
                  StructField("Quantity", StringType(),True),\
                  StructField("DUE",IntegerType(),True),\
                  StructField("GSTUS", IntegerType(),True),\
                  StructField("Product value",StringType(),True),\
                  StructField("Location",StringType(),True)\
                  StructField("Category",StringType(),True),\
                  StructField("product id",StringType(),True)
                   ])